In [81]:
import pandas as pd

In [82]:
engagement_dfs = []
for fold in [f"data/engagement/label_0402_fold_{i}" for i in range(5)]:
    train_df = pd.read_csv(f"{fold}/train.csv")
    val_df = pd.read_csv(f"{fold}/val.csv")
    engagement_dfs.extend([train_df, val_df])
engagement_dfs = pd.concat(engagement_dfs, ignore_index=True)
engagement_dfs = engagement_dfs.sort_values(by='video_path', ascending=True)
print(len(engagement_dfs['video_path']), len(engagement_dfs['video_path'].unique())) # TODO: ask TA wtf is this?
engagement_dfs = engagement_dfs.groupby('video_path').agg({
    'engagement': 'mean',
}).reset_index()
engagement_dfs['video_path'] = engagement_dfs['video_path'].apply(lambda x: f"data/engagement/video/{x}.mp4")
print(len(engagement_dfs))
engagement_dfs.head()

15665 3133
3133


,video_path,engagement
0,data/engagement/video/Ask_a_Therapist_How_to_M...,0.020000
1,data/engagement/video/Ask_a_Therapist_How_to_M...,0.100000
2,data/engagement/video/Ask_a_Therapist_How_to_M...,0.010000
3,data/engagement/video/Ask_a_Therapist_How_to_M...,0.146667
4,data/engagement/video/Ask_a_Therapist_How_to_M...,0.073333


In [83]:
df_video_path = pd.read_csv('data/engagement/raw_annotations_video_ids.csv')
print(len(df_video_path))
df_video_path.head()

19281


,Unnamed: 0,Vid_num,Video Link
0,0,0,NaN
1,1,1,data/engagement/video/Te_Awamutu_Community_Boa...
2,2,2,data/engagement/video/Group_therapy_video_2/cl...
3,3,3,data/engagement/video/Early_Mid_StageCare_Supp...
4,4,4,data/engagement/video/Stroke_Buddies_Support_G...


In [84]:
df_annotations = pd.read_csv('data/engagement/raw_annotations.csv')
print(len(df_annotations))
df_annotations.head()

19281


,Unnamed: 0,Annotator Name,Camera Question,Speaking Question,Quality Question,Q1,Q2,Q3,Q4,Q5,Q6,Vid_num
0,0,annotator_id_anonymized,No one is in the video or the camera is turned...,Is the person in the video the main speaker? (...,Is the video quality too poor to see the perso...,The listener produced verbal/audible sounds in...,The listener nodded or shook their head in res...,The listener made facial expressions in respon...,The listener was engaged in some other activit...,The listener looked away from the speaker. (e....,"Overall, would you say that the listener was s...",0
1,1,69,FALSE,No,No,Never,Never,Never,Never,Never,Yes,1
2,2,69,FALSE,No,No,Never,Rarely,Rarely,Never,Rarely,Yes,2
3,3,69,FALSE,No,No,Never,Never,Never,Rarely,Rarely,Unsure,3
4,4,69,FALSE,No,No,Sometimes,Often,Rarely,Sometimes,Sometimes,Unsure,4


In [85]:
df = pd.merge(
    df_video_path.drop(columns=['Unnamed: 0']),
    df_annotations.drop(columns=['Unnamed: 0', 'Annotator Name', 'Camera Question', 'Quality Question', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6']),
    how='inner',
    left_on='Vid_num',
    right_on='Vid_num'
).rename(columns={
    'Speaking Question': 'is_speaker',
    'Video Link': 'video_path',
})
# remove first row
df = df.iloc[1:]
# take the majority vote for is_speaker column to aggregate the labels
df['is_speaker'] = df['is_speaker'].apply(lambda x: 1 if str(x).lower() == 'yes' else 0)
df = df.groupby('video_path')['is_speaker'].agg(lambda x: 1 if sum(x) > len(x) / 2 else 0).reset_index()
df

,video_path,is_speaker
0,data/engagement/video/Ask_a_Therapist_How_to_M...,0
1,data/engagement/video/Ask_a_Therapist_How_to_M...,0
2,data/engagement/video/Ask_a_Therapist_How_to_M...,1
3,data/engagement/video/Ask_a_Therapist_How_to_M...,0
4,data/engagement/video/Ask_a_Therapist_How_to_M...,0
...,...,...
6395,data/engagement/video/zoom_group_therapy_sessi...,0
6396,data/engagement/video/zoom_group_therapy_sessi...,1
6397,data/engagement/video/zoom_group_therapy_sessi...,0
6398,data/engagement/video/zoom_group_therapy_sessi...,0


In [86]:
df = df.merge(
    engagement_dfs,
    how='left',
    left_on='video_path',
    right_on='video_path'
)
df

,video_path,is_speaker,engagement
0,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.020000
1,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.100000
2,data/engagement/video/Ask_a_Therapist_How_to_M...,1,NaN
3,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.010000
4,data/engagement/video/Ask_a_Therapist_How_to_M...,0,NaN
...,...,...,...
6395,data/engagement/video/zoom_group_therapy_sessi...,0,NaN
6396,data/engagement/video/zoom_group_therapy_sessi...,1,NaN
6397,data/engagement/video/zoom_group_therapy_sessi...,0,-0.040000
6398,data/engagement/video/zoom_group_therapy_sessi...,0,-0.350000


In [87]:
df['text_path'] = df['video_path'].apply(lambda x: x.replace('video', 'text').replace('.mp4', '.txt'))
df['audio_path'] = df['video_path'].apply(lambda x: x.replace('video', 'audio').replace('.mp4', '.wav'))
df['openface_path'] = df['video_path'].apply(lambda x: x.replace('video', 'featopenface').replace('.mp4', '.csv'))
df['clip_uid'] = df['video_path'].apply(lambda x: '_'.join(x.split('data/engagement/video/')[-1].split('.mp4')[0].split('/')[:-1]))
print(len(df))
df

6400


,video_path,is_speaker,engagement,text_path,audio_path,openface_path,clip_uid
0,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.020000,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...
1,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.100000,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...
2,data/engagement/video/Ask_a_Therapist_How_to_M...,1,NaN,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...
3,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.010000,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...
4,data/engagement/video/Ask_a_Therapist_How_to_M...,0,NaN,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...
...,...,...,...,...,...,...,...
6395,data/engagement/video/zoom_group_therapy_sessi...,0,NaN,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,zoom_group_therapy_session_1_clip0070
6396,data/engagement/video/zoom_group_therapy_sessi...,1,NaN,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,zoom_group_therapy_session_1_clip0070
6397,data/engagement/video/zoom_group_therapy_sessi...,0,-0.040000,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,zoom_group_therapy_session_1_clip0070
6398,data/engagement/video/zoom_group_therapy_sessi...,0,-0.350000,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,zoom_group_therapy_session_1_clip0070


In [88]:
df_listeners

,video_path,is_speaker,engagement,text_path,audio_path,openface_path,clip_uid
0,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.020000,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...
1,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.100000,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...
2,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.010000,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...
3,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.146667,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...
4,data/engagement/video/Ask_a_Therapist_How_to_M...,0,0.073333,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...
...,...,...,...,...,...,...,...
3128,data/engagement/video/zoom_group_therapy_sessi...,0,-0.276667,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,zoom_group_therapy_session_1_clip0069
3129,data/engagement/video/zoom_group_therapy_sessi...,0,-0.150000,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,zoom_group_therapy_session_1_clip0069
3130,data/engagement/video/zoom_group_therapy_sessi...,0,-0.040000,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,zoom_group_therapy_session_1_clip0070
3131,data/engagement/video/zoom_group_therapy_sessi...,0,-0.350000,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,zoom_group_therapy_session_1_clip0070


In [90]:
df_speakers = df[df['is_speaker'] == 1]
df_listeners = df[df['is_speaker'] == 0]

df_listener_speaker = df_listeners.merge(df_speakers, on='clip_uid', suffixes=('_listener', '_speaker'))

df_listener_speaker = df_listener_speaker[[
    'clip_uid',
    'video_path_listener', 'text_path_listener', 'audio_path_listener', 'openface_path_listener',
    'video_path_speaker', 'text_path_speaker', 'audio_path_speaker', 'openface_path_speaker',
    'engagement_listener'
]]

df_listener_speaker.columns = [
    'listener_clip_uid',
    'listener_video_path', 'listener_text_path', 'listener_audio_path', 'listener_openface_path',
    'speaker_video_path', 'speaker_text_path', 'speaker_audio_path', 'speaker_openface_path',
    'engagement'
]
df_listener_speaker

,listener_clip_uid,listener_video_path,listener_text_path,listener_audio_path,listener_openface_path,speaker_video_path,speaker_text_path,speaker_audio_path,speaker_openface_path,engagement
0,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,0.020000
1,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,0.100000
2,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,0.010000
3,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,NaN
4,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,NaN
...,...,...,...,...,...,...,...,...,...,...
4777,zoom_group_therapy_session_1_clip0069,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,-0.150000
4778,zoom_group_therapy_session_1_clip0070,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,NaN
4779,zoom_group_therapy_session_1_clip0070,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,-0.040000
4780,zoom_group_therapy_session_1_clip0070,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom

In [92]:
# drop those with NaN engagement
df_listener_speaker = df_listener_speaker.dropna(subset=['engagement'])
df_listener_speaker = df_listener_speaker.reset_index(drop=True)
df_listener_speaker.to_csv('engagement_listener_speaker.csv', index=False)
df_listener_speaker

,listener_clip_uid,listener_video_path,listener_text_path,listener_audio_path,listener_openface_path,speaker_video_path,speaker_text_path,speaker_audio_path,speaker_openface_path,engagement
0,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,0.020000
1,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,0.100000
2,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,0.010000
3,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,0.090000
4,Ask_a_Therapist_How_to_Manage_Mental_Health_Du...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,data/engagement/video/Ask_a_Therapist_How_to_M...,data/engagement/text/Ask_a_Therapist_How_to_Ma...,data/engagement/audio/Ask_a_Therapist_How_to_M...,data/engagement/featopenface/Ask_a_Therapist_H...,0.126667
...,...,...,...,...,...,...,...,...,...,...
2825,zoom_group_therapy_session_1_clip0069,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,-0.276667
2826,zoom_group_therapy_session_1_clip0069,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,-0.150000
2827,zoom_group_therapy_session_1_clip0070,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,data/engagement/video/zoom_group_therapy_sessi...,data/engagement/text/zoom_group_therapy_sessio...,data/engagement/audio/zoom_group_therapy_sessi...,data/engagement/featopenface/zoom_group_therap...,-0.040000
2828,zoom_group_therapy_session_1_clip0070,data/engagement/video/zoom_group_therapy_sessi...,data/enga